<a href="https://colab.research.google.com/github/esheshka/chess/blob/main/chess_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Globals**

table = [['.' for i in range(8)] for i in range(8)]
white_figures = ['P', 'R', 'N', 'B', 'Q', 'K']
black_figures = ['p', 'r', 'n', 'b', 'q', 'k']
figures = [*white_figures, *black_figures]
white_move = True
move_count = 1

castling_white = [True, True]
castling_black = [True, True]

In [ ]:
def create_table():
    global table
    global white_move
    global move_count
    global castling_white
    global castling_black

    table = [['.' for i in range(8)] for i in range(8)]

    table[0] = ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
    table[7] = ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']
    for x in range(8):
        table[1][x] = 'p'
        table[6][x] = 'P'

    table = flip(transpose(table))

    castling_white = [True, True]
    castling_black = [True, True]
    white_move = True
    move_count = 1

In [ ]:
def transpose(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[i][j]]
        res = res + [tmp]

    return res

In [ ]:
def flip(matr):
    res = []
    for j in range(8):
        tmp = []
        for i in range(8):
            tmp = tmp + [matr[j][7 - i]]
        res = res + [tmp]

    return res

In [ ]:
def draw():
    print(f"\n   A B C D E F G H\n")
    for x in range(8):
        print(8 - x, end='  ')

        for y in range(8):
            print(table[y][7 - x], end=' ')

        print('', 8 - x)

    print(f"\n   A B C D E F G H\n")

In [ ]:
def coordinates(text):
    result = [ord(text[0]) - ord('a'), int(text[1]) - 1]

    return result

In [ ]:
def move(command):
    global castling_white
    global castling_black
    global white_move
    global table

    if command == "0-0":
        if castling(True):
            return "The piece cannot make the specified move."
        else:
            return ''
    elif command == "0-0-0":
        if castling(False):
            return "The piece cannot make the specified move."
        else:
            return ''

    if len(command) != 5 or command[0] not in "abcdefghABSDEFGH" or command[1] not in "12345678" or command[2] != "-" or command[3] not in "abcdefghABSDEFGH" or command[4] not in "12345678":
        return "Wrong input format."

    move_from, move_to = map(coordinates, command.split("-"))

    if table[move_from[0]][move_from[1]] == ".":
        return "The piece cannot make the specified move."
    elif white_move and table[move_from[0]][move_from[1]] in black_figures:
        return "The piece cannot make the specified move."
    elif (not white_move) and table[move_from[0]][move_from[1]] in white_figures:
        return "The piece cannot make the specified move."

    possible = possible_moves(move_from[0], move_from[1])
    # print(possible)
    if move_to not in possible:
        return "The piece cannot make the specified move."
    else:
        if move_from == [4, 0] or move_to == [4, 0]:
            castling_white = [False, False]
        elif move_from == [0, 0] or move_to == [0, 0]:
            castling_white[1] = False
        elif move_from == [7, 0] or move_to == [7, 0]:
            castling_white[0] = False

        if move_from == [4, 7] or move_to == [4, 7]:
            castling_black = [False, False]
        elif move_from == [0, 7] or move_to == [0, 7]:
            castling_black[1] = False
        elif move_from == [7, 7] or move_to == [7, 7]:
            castling_black[0] = False

        table[move_to[0]][move_to[1]] = table[move_from[0]][move_from[1]]
        table[move_from[0]][move_from[1]] = '.'

In [ ]:
def castling(short):
    global castling_white
    global castling_black
    global white_move
    global table

    if short:
        if white_move and castling_white[0] and table[5][0] == '.' and table[6][0] == '.':
            table[5][0], table[6][0] = table[7][0], table[4][0]
            table[4][0], table[7][0] = '.', '.'
            castling_white = [False, False]
        elif (not white_move) and castling_black[0] and table[5][7] == '.' and table[6][7] == '.':
            table[5][7], table[6][7] = table[7][7], table[4][7]
            table[4][7], table[7][7] = '.', '.'
            castling_black = [False, False]
        else:
            return True
    else:
        if white_move and castling_white[1] and table[1][0] == '.' and table[2][0] == '.' and table[3][0] == '.':
            table[3][0], table[2][0] = table[0][0], table[4][0]
            table[4][0], table[0][0] = '.', '.'
            castling_white = [False, False]
        elif (not white_move) and castling_black[1] and table[1][7] == '.' and table[2][7] == '.' and table[3][7] == '.':
            table[3][7], table[2][7] = table[0][7], table[4][7]
            table[4][7], table[0][7] = '.', '.'
            castling_black = [False, False]
        else:
            return True

In [ ]:
def possible_moves(x, y):
    pos = []
    if table[x][y] in 'pP':
        pos = [*pos, *possible_moves_pawn(x, y)]
    if table[x][y] in 'rRqQ':
        pos = [*pos, *possible_moves_rook(x, y)]
    if table[x][y] in 'bBqQ':
        pos = [*pos, *possible_moves_bishop(x, y)]
    if table[x][y] in 'nN':
        pos = [*pos, *possible_moves_knight(x, y)]
    if table[x][y] in 'kK':
        pos = [*pos, *possible_moves_king(x, y)]


    return pos

In [ ]:
#@title **Возможные ходы короля**

def possible_moves_king(x, y):
    pos = []
    for x1 in [-1, 0, 1]:
        for y1 in [-1, 0, 1]:
            if (x1 == 0 and y1 == 0) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [ ]:
#@title **Возможные ходы коня**

def possible_moves_knight(x, y):
    pos = []
    for x1 in [-2, -1, 1, 2]:
        for y1 in [-2, -1, 1, 2]:
            if (x1*x1 + y1*y1 != 5) or (x + x1 not in range(8)) or (y + y1 not in range(8)):
                continue

            if table[x+x1][y+y1] == '.':
                pos.append([x+x1, y+y1])
            elif (table[x+x1][y+y1] in white_figures) == white_move:
                continue
            else:
                pos.append([x+x1, y+y1])

    return pos

In [ ]:
#@title **Возможные ходы слона**

def bishop_vectors(x, y, kx, ky):
    pos = []

    for a in range(1, 8):
        if y + ky*a not in range(8) or x + kx*a not in range(8):
            break
        elif table[x+kx*a][y+ky*a] == '.':
            pos.append([x+kx*a, y+ky*a])
        elif (table[x+kx*a][y+ky*a] in white_figures) == white_move:
            break
        else:
            pos.append([x+kx*a, y+ky*a])
            break

    return pos


def possible_moves_bishop(x, y):

    pos = []

    for x1 in [-1, 1]:
        for y1 in [-1, 1]:
            pos = [*pos, *rook_vectors(x, y, x1, y1)]

    return pos

In [ ]:
#@title **Возможные ходы ладьи**

def rook_vectors(x, y, kx, ky):
    pos = []

    for a in range(1, 8):
        if y + ky*a not in range(8) or x + kx*a not in range(8):
            break
        elif table[x+kx*a][y+ky*a] == '.':
            pos.append([x+kx*a, y+ky*a])
        elif (table[x+kx*a][y+ky*a] in white_figures) == white_move:
            break
        else:
            pos.append([x+kx*a, y+ky*a])
            break

    return pos


def possible_moves_rook(x, y):
    pos = []

    for x1 in [-1, 1]:
        pos = [*pos, *rook_vectors(x, y, x1, 0)]
    for y1 in [-1, 1]:
        pos = [*pos, *rook_vectors(x, y, 0, y1)]

    return pos

In [ ]:
#@title **Возможные ходы пешки**

def possible_moves_pawn(x, y):
    pos = []
    if white_move:
        if y+1 in range(8):
            if x+1 in range(8) and table[x+1][y+1] in black_figures:
                pos.append([x+1, y+1])
            if x-1 in range(8) and table[x-1][y+1] in black_figures:
                pos.append([x-1, y+1])

            if table[x][y+1] not in figures:
                pos.append([x, y+1])
                if y == 1 and table[x][y+2] not in figures:
                    pos.append([x, y+2])
    else:
        if y-1 in range(8):
            if x+1 in range(8) and table[x+1][y-1] in white_figures:
                pos.append([x+1, y-1])
            if x-1 in range(8) and table[x-1][y-1] in white_figures:
                pos.append([x-1, y-1])

            if table[x][y-1] not in figures:
                pos.append([x, y-1])
                if y == 6 and table[x][y-2] not in figures:
                    pos.append([x, y-2])

    return pos

In [ ]:
create_table()

while True:
    print('white'*white_move + 'black'*(not white_move), str(move_count) + ':')
    command = input()
    if command == 'exit':
        break
    elif command == 'draw':
        draw()
    else:
        error = move(command)
        if not error:
            white_move = not white_move
            if white_move:
                move_count += 1
        else:
            print('Error. Type:', error)

white 1:
exit


In [ ]:
# g2-g4
# g7-g5
# g1-f3
# g8-f6
# f1-g2
# f8-g7

False